# Normalize taxa list

Cleanup the normalized taxa list from the eODP researchers for a given particular taxon group. Create csvs to import the taxa data into the database.   

In [2]:
import sys

import pandas as pd
import numpy as np
import requests

sys.path.append('../../')
import scripts.normalize_taxa as nt

In [3]:
date = '2021-07-28'
input_file = f'../../raw_data/PI_processed_files/LIMS_Micropal_headers_PBDB_Taxonomy_notes_taxa_list_{date}.csv'

crosswalk_file = f"../../output/taxa/LIMS/taxa_crosswalk_{date}.csv"
taxa_list_file = f"../../output/taxa/LIMS/taxa_list_{date}.csv"

In [4]:
def log_df(df, row_count=5):
    print(df.shape)
    return df.head(row_count)


## Import normalized taxa list

In [46]:
df = pd.read_csv(input_file, skiprows = 9)
df = df.drop(df.index[[0, 1]])
df = df.dropna(how="all", axis="index")

log_df(df)

(4742, 21)


,taxon_group,verbatim_name,name,"name to use (if different from ""name"")",name comment field,Comment,notes,Any taxon above genus,genus modifier,genus name,...,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,comments,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
2,benthic_forams,Euuvigerina miozea (group) >100 m,Euuvigerina miozea (group) >100 m,Euuvigerina miozea,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,miozea,NaN,NaN,NaN,>100 m group,NaN,NaN,NaN
3,benthic_forams,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi (group) >50 m,Euuvigerina rodleyi,NaN,group,"Summer 2020: enter ""Euuvigerina miozea"" and re...",NaN,NaN,Euuvigerina,...,NaN,NaN,rodleyi,NaN,NaN,NaN,>50 m group,NaN,NaN,NaN
4,benthic_forams,Others,Others,Foraminifera indet.,NaN,not a taxa,Andy,Foraminifera indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,other benthic foraminifera,NaN,NaN,NaN
5,benthic_forams,Pleurostomellids comment,Pleurostomellids comment,Pleurostomellia indet.,NaN,not a taxa,Andy,Pleurostomellidae indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,benthic_forams,Ostracoda spp.,Ostracoda spp.,Ostracoda indet.,Ostracoda spp.,group abundance,"Summer 2020: go in as Ostracoda indet, and the...",Ostracoda indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df.shape

(4742, 21)

## Create crosswalk csv

In [47]:
fields = nt.taxa_rank_fields + nt.taxa_fields + nt.metadata_fields
filtered_taxa = pd.DataFrame(df, columns=fields)

set normalized_name using the taxa fields

In [48]:
nt.add_normalized_name_column(filtered_taxa)
log_df(filtered_taxa)

(4742, 14)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,comments
2,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,>100 m group
3,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,>50 m group
4,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,other benthic foraminifera
5,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,NaN
6,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams,Ostracoda spp.,NaN


In [49]:
filtered_taxa = filtered_taxa.drop(filtered_taxa[filtered_taxa['normalized_name'] == ''].index)
log_df(filtered_taxa)

(4735, 14)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,comments
2,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,>100 m group
3,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,>50 m group
4,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,other benthic foraminifera
5,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,NaN
6,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams,Ostracoda spp.,NaN


In [50]:
filtered_taxa.drop_duplicates(keep='first', inplace=True, subset =['verbatim_name', 'normalized_name'])
log_df(filtered_taxa)

(4594, 14)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,comments
2,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams,Euuvigerina miozea (group) >100 m,>100 m group
3,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams,Euuvigerina rodleyi (group) >50 m,>50 m group
4,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams,Others,other benthic foraminifera
5,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams,Pleurostomellids comment,NaN
6,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams,Ostracoda spp.,NaN


create crosswalk csv

In [15]:
filtered_taxa.to_csv(crosswalk_file, index=False)

## Create taxa list csv

create taxa list with unique taxa fields

In [51]:
fields = nt.taxa_rank_fields + nt.taxa_fields
taxa_df = pd.DataFrame(filtered_taxa, columns=fields)

# drop duplicate data
taxa_df = taxa_df.drop_duplicates() 

log_df(taxa_df)

(4209, 12)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group
2,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams
3,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams
4,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams
5,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams
6,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams


### create taxa csv

In [18]:
taxa_df.to_csv(taxa_list_file, index=False)

## create genera csv

In [4]:
taxa_df = pd.read_csv(taxa_list_file)
log_df(taxa_df)

(4209, 12)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group
0,NaN,NaN,Euuvigerina,NaN,NaN,NaN,miozea,NaN,NaN,NaN,Euuvigerina miozea,benthic_forams
1,NaN,NaN,Euuvigerina,NaN,NaN,NaN,rodleyi,NaN,NaN,NaN,Euuvigerina rodleyi,benthic_forams
2,Foraminifera indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foraminifera indet.,benthic_forams
3,Pleurostomellidae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pleurostomellidae indet.,benthic_forams
4,Ostracoda indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ostracoda indet.,benthic_forams


In [27]:
genus_df = pd.DataFrame(taxa_df['genus name'].str.strip().dropna().unique(), columns=['genus name']) 

log_df(genus_df)

(1015, 1)


,genus name
0,Euuvigerina
1,Nodosaria
2,Cibicides
3,Brizalina
4,Candeina


In [1]:
PBDB_API = "https://paleobiodb.org/data1.2/"
PBDB_TAXA = f"{PBDB_API}taxa/single.json?vocab=pbdb&name="

In [30]:
for index, row in genus_df.iterrows():
    if index % 50 == 0:
        print(index, end=' ')
        
    url =  PBDB_TAXA +  row['genus name']
    if row['genus name'] == 'Martinotiella':
        url =  PBDB_TAXA + 'Martinottiella'
        
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            # cast taxon_no to string to avoid pandas converting it to a float           
            genus_df.at[index, 'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            genus_df.at[index, 'pbdb_taxon_name'] = data[0]["taxon_name"]
            genus_df.at[index, 'pbdb_taxon_rank'] = data[0]["taxon_rank"]

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 

In [32]:
genus_df.to_csv(f"../../output/taxa/LIMS/genera_{date}.csv", index=False)

In [31]:
genus_df

,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank
0,Euuvigerina,1408,Euuvigerina,genus
1,Nodosaria,1952,Nodosaria,genus
2,Cibicides,1107,Cibicides,genus
3,Brizalina,1017,Brizalina,genus
4,Candeina,1053,Candeina,genus
...,...,...,...,...
1010,Valkyria,432809,Valkyria,genus
1011,Velicucullus,705,Velicucullus,genus
1012,Zealithapium,84700,Zealithapium,genus
1013,Zygocircus,714,Zygocircus,genus
